# Modeling and Simulation in Python

Starter code for the orbit example

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

### Earth falling into the sun

Here's a question from the web site [Ask an Astronomer](http://curious.astro.cornell.edu/about-us/39-our-solar-system/the-earth/other-catastrophes/57-how-long-would-it-take-the-earth-to-fall-into-the-sun-intermediate):

"If the Earth suddenly stopped orbiting the Sun, I know eventually it would be pulled in by the Sun's gravity and hit it. How long would it take the Earth to hit the Sun? I imagine it would go slowly at first and then pick up speed."

Here's a solution.

In [2]:
# Here are the units we'll need

s = UNITS.second
N = UNITS.newton
kg = UNITS.kilogram
m = UNITS.meter

<Unit('meter')>

In [3]:
# And an inition condition (with everything in SI units)
r_0 = 147e9 * m
x = 147e9 * m
y = 0 * m
vx = 0 * m /s
vy = 30330 * m /s            
             
init = State(r = Vector (x, y),
             v = Vector (vx, vy))

,values
r,"[147000000000.0 meter, 0.0 meter]"
v,"[0.0 meter / second, 30330.0 meter / second]"


In [4]:
# Making a system object

r_earth = 6.371e6 * m
r_sun = 695.508e6 * m

system = System(init=init,
                G=6.674e-11 * N / kg**2 * m**2,
                m1=1.989e30 * kg,
                r_final=r_sun + r_earth,
                m2=5.972e24 * kg,
                t_0=0 * s,
                t_end=1e7 * s)

,values
init,"r [147000000000.0 meter, 0.0 met..."
G,6.674e-11 meter ** 2 * newton / kilogram ** 2
m1,1.989e+30 kilogram
r_final,701879000.0 meter
m2,5.972e+24 kilogram
t_0,0 second
t_end,10000000.0 second


In [5]:
# Here's a function that computes the force of gravity

def universal_gravitation(state, system):
    """Computes gravitational force.
    
    state: State object with distance r
    system: System object with m1, m2, and G
    """
    r, v = state
    unpack(system)
    
    a, b = pol2cart (r.angle, ((G * m1 * m2) / (x**2 + y **2)))
    force = Vector (a, b)
    return force

In [6]:
universal_gravitation(init, system)

<Quantity([3.6686486e+22 0.0000000e+00], 'newton')>

In [7]:
# The slope function

def slope_func(state, t, system):
    """Compute derivatives of the state.
    
    state: position, velocity
    t: time
    system: System object containing `g`
    
    returns: derivatives of y and v
    """
    r, v = state
    unpack(system)    

    force = universal_gravitation(state, system)
    dxdt = vx
    dydt = vy
    dvxdt = -force.angle / m2
    dvydt = - (force - force * (force.angle)) / m2
    
    return dxdt, dydt, dvxdt, dvydt

In [8]:
# Always test the slope function!

slope_func(init, 0, system)

(<Quantity(0.0, 'meter / second')>,
 <Quantity(30330.0, 'meter / second')>,
 <Quantity(-0.0, 'radian / kilogram')>,
 <Quantity([-0.00614308 -0.        ], 'newton / kilogram')>)

In [9]:
# Here's an event function that stops the simulation
# before the collision

def event_func(state, t, system):
    r, v = state
    return r - system.r_final

In [10]:
# Always test the event function!

event_func(init, 0, system)

<Quantity([ 1.46298121e+11 -7.01879000e+08], 'meter')>

In [11]:
# Finally we can run the simulation
results, details = run_ode_solver(system, slope_func, events=event_func)
details

ValueError: `y0` must be 1-dimensional.

In [ ]:
# Here's how long it takes...

t_final = get_last_label(results) * s

In [ ]:
# ... expressed in units we understand

t_final.to(UNITS.day)

In [ ]:
# Before plotting, we run the simulation again with `t_eval`

ts = linspace(t_0, t_final, 201)
results, details = run_ode_solver(system, slope_func, events=event_func, t_eval=ts)

In [ ]:
# Scaling the time steps to days

results.index /= 60 * 60 * 24

In [ ]:
# Scaling the distance to million km

r = results.r / 1e9;

In [ ]:
# And plotting

plot(r, label='r')

decorate(xlabel='Time (day)',
         ylabel='Distance from sun (million km)')